In [1]:
!pip install --upgrade albumentations -q
!pip install gdown

import os
import pandas as pd
import pickle
import shutil
import numpy as np
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import ImageFile
import glob
import shutil
import cv2

from sklearn.datasets import load_files
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import utils
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image  

import albumentations as A
from tensorflow.keras.models import Model
import tensorflow.keras.layers
from tensorflow.keras.layers import Dense, ReLU
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.models import load_model

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9681 sha256=937dff78f1a7a62af1d810ebf9f523989541e55dc7e5813fce35e4db67f40818
  Stored in directory: /root/.cache/pip/wheels/ba/e0/7e/726e872a53f7358b4b96a9975b04e98113b005cd8609a63abc
Successfully built gdown


In [4]:
def draw_plot_from_history(history, metric, n_epochs, stage):
    plt.style.use("ggplot")
    plt.figure(figsize=(9, 5))
    plt.plot(np.arange(0, n_epochs), history.history[metric], label="train_%s" % metric)
    if "val_%s" % metric in history.history:
        plt.plot(np.arange(0, n_epochs), history.history["val_%s" % metric], label="val_%s" % metric)
    title = stage + '_%s' % metric
    plt.title(title)
    plt.xlabel("Epoch #")
    plt.ylabel("Value")
    plt.legend()
    filename = title + '.png'
    plt.show()

In [5]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data, batch_size, augmentation, image_size, shuffle=True):
        self.batch_size = batch_size
        self.augmentation = augmentation
        self.shuffle = shuffle
        self.image_size = image_size

        self.image_names = np.array([item['path'] for item in data])
        self.targets = to_categorical(np.array([int(item['label'][1:]) for item in data]))
        self.samples = len(self.targets)

        self.indexes = np.arange(self.samples)
        if shuffle:
            np.random.shuffle(self.indexes)

    def __len__(self):
        return int(np.ceil(self.samples / self.batch_size))
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def transform_image(self, image):
        image = cv2.resize(image, (self.image_size, self.image_size), interpolation = cv2.INTER_CUBIC)

        if self.augmentation:
            image = self.augmentation(image=image)['image']

        return image

    def __getitem__(self, index):
        take_ind = self.indexes[index * self.batch_size: min((index + 1) * self.batch_size, len(self.targets))]
        X = np.empty((len(take_ind), self.image_size, self.image_size, 3))
        y = self.targets[take_ind, :]

        for i in range(len(take_ind)):
            img = cv2.imread(self.image_names[take_ind[i]], cv2.IMREAD_COLOR)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = self.transform_image(img)
            X[i] = img

        return X, y

Create augmentations:

In [9]:
transform = A.Compose(
    [
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=30, 
                           p=0.5, border_mode=cv2.BORDER_CONSTANT), 
        A.RandomBrightnessContrast(p=0.5),
        A.RGBShift(p=0.25),
        A.GaussNoise(p=0.25),
        A.HorizontalFlip(p=0.5),
    ]
)

Create generators:

In [6]:
TRAIN_DIR = "/kaggle/input/state-farm-distracted-driver-detection/imgs/train"
TEST_DIR = "/kaggle/input/state-farm-distracted-driver-detection/imgs/test"

In [7]:
INPUT_SHAPE = 380
BATCH_SIZE = 32
num_classes = 10

In [10]:
train_data = []
label_stat = []

for label in os.listdir(TRAIN_DIR):
    for img_path in glob.glob(os.path.join(TRAIN_DIR, label, "*.jpg")):
        train_data.append({'path': img_path, 'label': label})
        label_stat.append(label)

train_data, val_data = train_test_split(train_data, test_size=0.2, stratify=label_stat, shuffle=True)

train_generator = DataGenerator(train_data, BATCH_SIZE, transform, INPUT_SHAPE)
validation_generator = DataGenerator(val_data, BATCH_SIZE, None, INPUT_SHAPE, shuffle=False)

In [ ]:
# train_generator.augmentation = transform

Build model:

In [ ]:
model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(INPUT_SHAPE, INPUT_SHAPE, 3))

for layer in model.layers:
    layer.trainable = False

x = model.output
x = GlobalAveragePooling2D()(x)
# x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=model.input, outputs=predictions)

Warmup:

In [ ]:
NUM_EPOCHS = 5
LEARNING_RATE = 0.001

In [ ]:
opt = Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=["accuracy"])

history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // BATCH_SIZE,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples // BATCH_SIZE,
    epochs=NUM_EPOCHS, verbose=1)

draw_plot_from_history(history, 'loss', NUM_EPOCHS, 'STAGE_1')
draw_plot_from_history(history, 'accuracy', NUM_EPOCHS, 'STAGE_1')

In [ ]:
model.save("model1_stage1.hdf5")

Unfreezeing deeper layers:

In [ ]:
NUM_EPOCHS = 30
LEARNING_RATE = 0.0003
FINE_TUNE_FROM_LAYER = -20

In [ ]:
for layer in model.layers:
    layer.trainable = False
for layer in model.layers[FINE_TUNE_FROM_LAYER:]:
    if not isinstance(layer, tensorflow.keras.layers.BatchNormalization):
        layer.trainable = True

In [ ]:
learning_rate_fn = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=LEARNING_RATE,
    decay_steps=NUM_EPOCHS * train_generator.samples // BATCH_SIZE,
    end_learning_rate=LEARNING_RATE / 10,
    power=1.0)

opt = Adam(learning_rate=learning_rate_fn)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=["accuracy"])

save_best_checkpoint = ModelCheckpoint("model1_stage2.hdf5", verbose=1, monitor='val_loss', save_best_only=True, mode='auto')

history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // BATCH_SIZE,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples // BATCH_SIZE,
    epochs = NUM_EPOCHS,
    callbacks=[save_best_checkpoint], 
    verbose=1)

draw_plot_from_history(history, 'loss', NUM_EPOCHS, 'STAGE_2')
draw_plot_from_history(history, 'accuracy', NUM_EPOCHS, 'STAGE_2')

In [ ]:
def save_plot_from_history(history, metric, n_epochs, stage):
    plt.style.use("ggplot")
    plt.figure(figsize=(21, 15))
    plt.plot(np.arange(0, n_epochs), history.history[metric], label="train_%s" % metric)
    if "val_%s" % metric in history.history:
        plt.plot(np.arange(0, n_epochs), history.history["val_%s" % metric], label="val_%s" % metric)
    title = metric
    plt.title(title, fontsize=18)
    plt.xlabel("Epoch #", fontsize=18)
    plt.ylabel("Value", fontsize=18)
    plt.legend(fontsize=18)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    filename = title + '.png'
    plt.savefig(filename)

In [ ]:
save_plot_from_history(history, 'loss', NUM_EPOCHS, 'STAGE_2')
save_plot_from_history(history, 'accuracy', NUM_EPOCHS, 'STAGE_2')

Make predictions:

In [ ]:
from tqdm.auto import tqdm

In [ ]:
def classify(model, test_img_dir):

    result = dict()
    keys = ['name', *['c%d' % i for i in range(10)]]
    for key in keys:
        result[key] = []
        
    paths = sorted(list(os.listdir(test_img_dir)))

    for path in tqdm(paths):
        if not path.endswith('.jpg'):
            continue
        
        image = cv2.imread(os.path.join(test_img_dir, path), cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (INPUT_SHAPE, INPUT_SHAPE), interpolation = cv2.INTER_CUBIC)

        pred = model.predict(np.expand_dims(image, 0))[0]
        result['name'].append(path)
        for i in range(10):
            result['c%d' % i].append(pred[i])

    return result

In [ ]:
model = load_model('model1_stage2.hdf5')

In [ ]:
result = classify(model, TEST_DIR)

In [ ]:
dct = {'img': result['name']}
for i in range(10):
    col = 'c%d' % i
    dct[col] = result[col]
    
df = pd.DataFrame(dct)
df.to_csv('submission.csv', index=False)
df

Improve score a bit with clipping hack:

In [ ]:
df_new = df.copy()

cols = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
arr, names = df_new.values[:, 1:], df_new.values[:, 0]
arr = np.clip(arr, 0.025, 0.975)
arr /= np.sum(arr, axis=1, keepdims=True)

new_df = pd.DataFrame(np.hstack([names.reshape((-1, 1)), arr]), columns=df.columns)
new_df.to_csv('submission_new.csv', index=False)

Extract logits on train and val for distillation:

In [17]:
batch_ind, num_batches = None, None

In [42]:
class PredictDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data, batch_size, augmentation, image_size):
        self.batch_size = batch_size
        self.augmentation = augmentation
        self.image_size = image_size

        self.image_names = np.array([item['path'] for item in data])
        self.targets = to_categorical(np.array([int(item['label'][1:]) for item in data]))
        self.samples = len(self.targets)

        self.indexes = np.arange(self.samples)

    def __len__(self):
        return int(np.ceil(self.samples / self.batch_size))

    def transform_image(self, image):
        image = cv2.resize(image, (self.image_size, self.image_size), interpolation = cv2.INTER_CUBIC)

        if self.augmentation:
            image = self.augmentation(image=image)['image']

        return image

    def __getitem__(self, index):
        global batch_ind, num_batches
        if batch_ind % 10 == 0:
            print(f"{batch_ind + 1} / {num_batches}")
        batch_ind += 1
        
        take_ind = self.indexes[index * self.batch_size: min((index + 1) * self.batch_size, len(self.targets))]
        X = np.empty((len(take_ind), self.image_size, self.image_size, 3))
        y = self.targets[take_ind, :]

        for i in range(len(take_ind)):
            img = cv2.imread(self.image_names[take_ind[i]], cv2.IMREAD_COLOR)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = self.transform_image(img)
            X[i] = img

        return X, y

In [43]:
import gdown

url = 'https://drive.google.com/u/0/uc?id=1iMZeYCX-Qijk4iPL5AULQvt21LroUw3K&export=download'
output = 'teacher.zip'
gdown.download(url, output, quiet=False)
!unzip teacher.zip

teacher = load_model('model1_stage2.hdf5')

teacher_copy= tf.keras.models.clone_model(teacher)
teacher_copy.build(teacher.input_shape)
teacher_copy.set_weights(teacher.get_weights())

x = teacher_copy.layers[-2].output
predictions = Dense(num_classes, activation='linear')(x)
teacher_copy = Model(inputs=teacher_copy.input, outputs=predictions)
teacher_copy.layers[-1].set_weights(teacher.layers[-1].get_weights())

model = teacher_copy
model._name = 'teacher'

In [45]:
import pickle

BATCH_SIZE = 32
INPUT_SHAPE = 380

predict_train_generator = PredictDataGenerator(train_data, BATCH_SIZE, None, INPUT_SHAPE)

batch_ind, num_batches = 0, np.ceil(predict_train_generator.samples / BATCH_SIZE).astype(int)

filenames = predict_train_generator.image_names
predict = model.predict_generator(predict_train_generator, steps=num_batches)

train_pred = (filenames, predict)
pickle.dump(train_pred, open('train_pred', 'wb'))

In [46]:
import pickle

BATCH_SIZE = 32
INPUT_SHAPE = 380

predict_val_generator = PredictDataGenerator(val_data, BATCH_SIZE, None, INPUT_SHAPE)

batch_ind, num_batches = 0, np.ceil(predict_val_generator.samples / BATCH_SIZE).astype(int)

filenames = predict_val_generator.image_names
predict = model.predict_generator(predict_val_generator, steps=num_batches)

val_pred = (filenames, predict)
pickle.dump(val_pred, open('val_pred', 'wb'))

1 / 141
11 / 141
21 / 141
31 / 141
41 / 141
51 / 141
61 / 141
71 / 141
81 / 141
91 / 141
101 / 141
111 / 141
121 / 141
131 / 141
141 / 141
